In [10]:
import requests
import json
from pandas import json_normalize
import pandas as pd

# Count the number of HPO ID
### total 16480

In [4]:
from owlready2 import *
onto = get_ontology("http://purl.obolibrary.org/obo/hp.owl").load()
obo = onto.get_namespace("http://purl.obolibrary.org/obo/")
ontology_classes = obo.HP_0000001.descendants()

i = 0
for current_class in ontology_classes:
    hp_id = current_class.name.replace('_', ':')
    i = i + 1
print(str(i))

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/BFO_0000023
  http://purl.obolibrary.org/obo/CHEBI_50906

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/CHEBI_36080
  http://purl.obolibrary.org/obo/PR_000000001



16480


# Find descendants of Muscular dystrophy

In [6]:
from pyhpo import Ontology
_ = Ontology()
term = Ontology.get_hpo_object('HP:0003560')
for p in term.children:
    print(p)

HP:0007081 | Late-onset muscular dystrophy
HP:0006785 | Limb-girdle muscular dystrophy
HP:0003741 | Congenital muscular dystrophy
HP:0008970 | Scapulohumeral muscular dystrophy


# Co-occurence & Jaccard Index

In [7]:
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': ['90006785; 90007081; 90008970; 90003741']}
response = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query)

info = response.json()
df = json_normalize(info['results'])
df['Jaccard'] = df['concept_pair_count']/(df['concept_count_1']+df['concept_count_2']-df['concept_pair_count']) 
df_sorted = df.sort_values(by=['concept_pair_count'], ascending= False)
n_df = df_sorted.loc[:, ("concept_name_2", "concept_pair_count", "Jaccard")]
n_df

,concept_name_2,concept_pair_count,Jaccard
0,Limb-girdle muscular dystrophy,315,0.191257
1,Congenital muscular dystrophy,174,0.106552
2,Late-onset muscular dystrophy,22,0.013631


# With All HPO Terms

In [7]:
o_list = [] 
for current_class in ontology_classes:
    hp_id = current_class.name.replace('_', ':')
    o_list.append(hp_id)

16480

In [20]:
o_100 = o_list[0:100]
s = ",".join(o_100).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
result = json_normalize(info['results'])

In [21]:
i = 101
while i < 16400: 
    o_100 = o_list[i:i+99]
    s = ",".join(o_100).replace(",", ";")
    para = {'q': s}
    response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
    df_z = json_normalize(response['results'])
    list_z = df_z['concept_id'].values.tolist()
    list_z = list(map(str, list_z))
    str_z = ",".join(list_z).replace(",", ";")
    query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
    info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
    df = json_normalize(info['results'])
    result = pd.concat([result, df])
    i = i+100
    try:
        info is None
    except:
        pass
    

In [23]:
o_100 = o_list[16401:16480]
s = ",".join(o_100).replace(",", ";")
para = {'q': s}
response = requests.get('https://rare.cohd.io/api/vocabulary/findConceptByCode', params = para).json()
df_z = json_normalize(response['results'])
list_z = df_z['concept_id'].values.tolist()
list_z = list(map(str, list_z))
str_z = ",".join(list_z).replace(",", ";")
query = {'dataset_id': 2, 'concept_id_1':'90003560', 'concept_id_2': str_z}
info = requests.get('https://rare.cohd.io/api/frequencies/pairedConceptFreq', params=query).json()
df1 = json_normalize(info['results'])
result = pd.concat([result, df1])
result = result.drop_duplicates(['concept_name_2'])
result['Jaccard'] = result['concept_pair_count']/(result['concept_count_1']+result['concept_count_2']-result['concept_pair_count'])
result = result.sort_values(by=['Jaccard'], ascending=False)
result = result.loc[:,("concept_name_2", "Jaccard", "concept_count_2", "concept_pair_count", "concept_id_2" )]
result.head(10)


,concept_name_2,Jaccard,concept_count_2,concept_pair_count,concept_id_2
0,Limb-girdle muscular dystrophy,0.191257,351,315,90006785
0,Congenital muscular dystrophy,0.106552,196,174,90003741
0,Progressive muscle weakness,0.074058,811,167,90003323
0,Increased variability in muscle fiber diameter,0.072464,1275,195,90003557
0,Myopathy,0.066953,8078,608,90003198
0,Rimmed vacuoles,0.053993,4948,336,90003805
1,Proximal muscle weakness,0.053615,1828,175,90003701
2,Gowers sign,0.049487,128,82,90003391
2,Calf muscle hypertrophy,0.040244,95,66,90008981
3,Inflammatory myopathy,0.039742,1293,111,90009071


### Limb-girdle muscular dystrophy, Congenital muscular dystrophy, Progressive muscle weakness, Increased variability in muscle fiber diameter, Myopathy, Rimmed vacuoles, Proximal muscle weakness, Gowers sign, Calf muscle hypertrophy, inflammatory myopathy are the top 10 Jaccard index